In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [189]:
#ваш код здесь
df_hh = pd.read_csv('data/dst-3.0_16_1_hh_database.csv', sep=';')
df_hh_copy = df_hh.copy()


In [191]:
df_hh_copy['ЗП'].apply(lambda x: x.split()[1].replace('.', '')).unique()

array(['руб', 'белруб', 'KZT', 'EUR', 'USD', 'грн', 'сум', 'KGS', 'AZN'],
      dtype=object)

In [3]:
df_hh_copy['образование'] = df_hh_copy['Образование и ВУЗ'].apply(lambda x: x[:x.find('образование')].lower().strip())
print(f"Колличество сосискателей со средним образованием: {df_hh_copy['образование'].value_counts().loc['среднее']}")

Колличество сосискателей со средним образованием: 559


In [4]:
df_hh_copy['Пол'] = df_hh_copy['Пол, возраст'].apply(lambda x: x[0])
df_hh_copy['Возраст'] = df_hh_copy['Пол, возраст'].apply(lambda x: int(x.split(',')[1].split()[0]))


In [5]:
#
def experience(arg):
    if arg is np.nan or arg == 'Не указано':
        return None
    lst_month = ['месяц', 'месяца', 'месяцев']
    lst_year = ['год', 'года', 'лет']
    month = 0
    lst = arg.split()
    for item, elem in enumerate(lst[:7]):
        if elem in lst_month:
            month += int(lst[item - 1])
        elif elem in lst_year:
            month += int(lst[item - 1]) * 12
    
    return month

df_hh_copy['Опыт работы (месяцев)'] = df_hh_copy['Опыт работы'].apply(experience).astype('Int64')  
df_hh_copy['Опыт работы (месяцев)'].median()
    

100.0

In [ ]:
df_hh_copy['Город, переезд, командировки'].apply(lambda x: x.split(',')[1]).unique()

In [16]:
#df_hh_copy['Готовность к переезду'] = df_hh_copy['Город, переезд, командировки'].apply(lambda x: x.find('перее') != -1 & x.find('не') != -1)
#df_hh_copy['Готовность к командировкам'] = df_hh_copy['Город, переезд, командировки'].apply(lambda x: x.find('командир') != -1 & x.find('не') != -1)
mask = df_hh_copy['Готовность к командировкам']
mask2 = df_hh_copy['Готовность к переезду']
percent = round((df_hh_copy[mask & mask2].shape[0] * 100) / df_hh_copy.shape[0], 2 )
print(f"доля соискателей готовых одновременно и к переездам, и к командировкам: {percent}%")

доля соискателей готовых одновременно и к переездам, и к командировкам: 32.04%


In [144]:
df_hh_copy = df_hh.copy()

In [209]:
def removal(arg):
    f = arg.split(',')
    for elem in f:
        if 'перее' in elem:
            if 'не' not in elem:
                return True
    return False
    

df_hh_copy['Готовность к переезду'] = df_hh_copy['Город, переезд, командировки'].apply(removal)
#df_hh_copy.drop(['Готовность к переезду'], axis=1, inplace=True)
df_hh_copy['Готовность к переезду'].value_counts()

False    28741
True     16003
Name: Готовность к переезду, dtype: int64

In [208]:
def business_travel(arg):
    f = arg.split(',')
    for elem in f:
        if 'команд' in elem:
            if 'не' not in elem:
                return True
            
    return False
df_hh_copy['Готовность к командировкам'] = df_hh_copy['Город, переезд, командировки'].apply(business_travel)
#df_hh_copy.drop(['Готовность к переезду'], axis=1, inplace=True)
df_hh_copy['Готовность к командировкам'].value_counts()

True     31643
False    13101
Name: Готовность к командировкам, dtype: int64

In [172]:
df_hh_copy['R'] = df_hh_copy['Город, переезд, командировки'].apply(lambda x: ', '.join(x.split(',')[1:3]))
df_hh_copy.groupby(['R'], as_index=False)[['Готовность к переезду', 'R', 'Готовность к командировкам']].tail(10)

,Готовность к переезду,R,Готовность к командировкам
8,True,"м. Шоссе Энтузиастов , готов к переезду",True
19,True,"м. Речной вокзал , готов к переезду (Москва)",True
26,True,"м. Молодежная , готова к переезду (Санкт-Пе...",True
30,True,"м. Кунцевщина , готова к переезду",True
60,True,"готов к переезду (Новосибирск) , не готов к ...",False
...,...,...,...
44739,True,"готов к переезду (Москва, Химки)",True
44740,True,"готов к переезду , готов к командировкам",True
44741,True,"готова к переезду , готова к редким командир...",True
44742,False,"не готов к переезду , не готов к командировкам",False


In [210]:
mask = df_hh_copy['Готовность к командировкам']
mask2 = df_hh_copy['Готовность к переезду']
percent = round((df_hh_copy[mask & mask2].shape[0] * 100) / df_hh_copy.shape[0], 2 )
print(f"доля соискателей готовых одновременно и к переездам, и к командировкам: {percent}%")

доля соискателей готовых одновременно и к переездам, и к командировкам: 31.83%


In [77]:
def cities(arg):
    f = arg.split()[0]
    
    million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    for city in million_cities:
        if city in f:
            return "город-миллионник"
        elif f in 'Москва' or 'Санкт-Петербург':
            return f
        else:
            return 'другие'
        
df_hh_copy['Город'] = df_hh_copy['Город, переезд, командировки'].apply(cities)
#df_hh_copy.drop(['Город'], axis=1, inplace=True)
print(f"Доля сосискателей живущих в Санкт-Петербурге: {round(df_hh_copy['Город'].value_counts(normalize=True).loc['Санкт-Петербург'] * 100)}")
df_hh_copy['Город'].nunique()

Доля сосискателей живущих в Санкт-Петербурге: 11


958

In [195]:
#df_hh_copy = df_hh.copy()
df_hh_copy['Обновление резюме'] = pd.to_datetime(df_hh_copy['Обновление резюме'], dayfirst=True).dt.date


In [196]:
def currency(arg):
    arg = arg.replace('.', '')
    cur = arg.split()
    dic = {'грн':'UAH','белруб':'BYN','сум':'UZS'}
    return dic.get(cur[1], cur[1])
    
    
df_hh_copy['currency'] = df_hh_copy['ЗП'].apply(currency)
df_hh_copy['ЗП'] = df_hh_copy['ЗП'].apply(lambda x : int(x.split()[0]))


In [178]:
df_hh_copy['currency'].unique()

array(['руб', 'BYN', 'KZT', 'EUR', 'USD', 'UAH', 'UZS', 'KGS', 'AZN'],
      dtype=object)

In [197]:
df = pd.read_csv('data/ExchangeRates.csv')
df['date'] = pd.to_datetime(df['date'], dayfirst=True).dt.date


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5664 entries, 0 to 5663
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   currency    5664 non-null   object 
 1   per         5664 non-null   object 
 2   date        5664 non-null   object 
 3   time        5664 non-null   object 
 4   close       5664 non-null   float64
 5   vol         5664 non-null   int64  
 6   proportion  5664 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 309.9+ KB


In [199]:
df_hh_copy = df_hh_copy.merge(df,how='left', left_on=['Обновление резюме', 'currency'], right_on=['date','currency'])

In [200]:
df_hh_copy = df_hh_copy.fillna({'close':1, 'proportion':1})
df_hh_copy['proportion'].astype('int64').unique()


array([    1,   100,    10, 10000], dtype=int64)

In [ ]:
df_hh_copy['ЗП'] = df_hh_copy['ЗП'].astype('Int64')
df_hh_copy['proportion'] = df_hh_copy['proportion'].astype('Int64')
df_hh_copy['ЗП (руб)'] = (df_hh_copy['ЗП'] / df_hh_copy['proportion']) * df_hh_copy['close']
df_hh_copy[df_hh_copy['currency'] == 'UZS']

In [202]:
round(df_hh_copy['ЗП (руб)'].median() / 1000)

59

In [211]:
mask = df_hh_copy['ЗП (руб)'] < 1e6
groub = df_hh_copy.groupby(['Готовность к переезду', 'Готовность к командировкам'], as_index=False)['ЗП (руб)'].median()
groub

,Готовность к переезду,Готовность к командировкам,ЗП (руб)
0,False,False,40000.0
1,False,True,60000.0
2,True,False,50000.0
3,True,True,66000.0


In [212]:


fig = px.bar(
    groub,
    x='Готовность к переезду',
    y='ЗП (руб)',
    color='Готовность к командировкам',
    barmode='group'
)
fig.show()